# 

In [36]:
import pandas as pd
from bioinfokit.analys import Fasta
import os
# helpers function

def get_len_fasta(filename):
    counter = {}
    with open(filename) as f:
        te_name = ""
        for line in f.readlines():
            if line.startswith('>'):
                te_name = line[1:].strip()
                counter[te_name] = 0
            else:
                if te_name:
                    counter[te_name] = counter[te_name] + len(line) - line.count("~")
    return counter

def get_from_te(fasta, te_name):
    te_len = get_len_fasta(fasta)[te_name]
    return int(te_len - te_len * dif_percent / 100)

def get_to_te(fasta, te_name):
    te_len = get_len_fasta(fasta)[te_name]
    return int(te_len + te_len * dif_percent / 100)

def read_fasta(filename):
    return pd.read_csv(filename, sep="\t", header=None).rename(headers_dict, axis=1)

def save_fasta(df, filename):
    df.to_csv(filename, sep='\t', index=False, encoding='utf-8', header=False)

def _create_part_raport(df):
    # there is one qseqid
    # it should be sortet to q_start
    df = df.sort_values(by=['sseqid', 'qstart', 'qend', 'sstart', 'send'])
    start = df[:-1].reset_index(drop=True)
    end = df[1:].reset_index(drop=True)
    return pd.DataFrame(
        {
            'qseqid':df.qseqid.values[0],
            'sseqid':df.sseqid.values[0],
            'start_te': start.qend,
            'end_te': end.qstart,
            'start_genome_te': start.send,
            'end_genome_te': end.sstart,

            
            
            'qstart1': end.qstart,
            'qend1': end.qend,
            'sstart1': end.sstart,
            'send1': end.send,
            
            'qstart2': start.qstart,
            'qend2': start.qend,
            'sstart2': start.sstart,
            'send2': start.send,

            'length': end.qstart - start.qend,
            'length_genome': end.sstart - start.send,
        }
    )

def create_report(df):
    series_list = []
    for qseqid in df.qseqid.unique():
        okno = df[df.qseqid == qseqid]
        for sseqid in okno.sseqid.unique():
            okno2 = okno[(okno.sseqid == sseqid)]
            series_list.append(_create_part_raport(okno2))
    return pd.concat(series_list)

def filter_finish(df):
    return df[
        (df.length < to_te)
        & (df.length > from_te)
        & (df.length_genome > -100)
        & (df.length_genome < 12000)
    ]


In [40]:
ont = ["K10B", "K10f", "LK_1.3"]
genome_path = "genome/DCARv3.4.fa"
te_path = "te/Alex1_3.fas"
dif_percent = 30

te_name = 'Alex1_chr1:18795335-18799058'
te_name = 'Alex1'
te_len = get_len_fasta(te_path)[te_name]
from_te = get_from_te(te_path, te_name)
to_te = get_to_te(te_path, te_name)


In [43]:
get_len_fasta('te/Alex1_3.fas')

{'Alex1': 4807, 'Alex3': 4847}

In [42]:
def mask_genome(te_name, te_path, genome_path, make_db=False):
    os.system(f"mkdir -p masked_genome")
    if make_db:
        os.system(f"makeblastdb -in {genome_path} -dbtype nucl -out genome/genome")
    os.system(f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db genome/genome -out masked_genome/{te_name}_genome.bl -perc_identity 0.9")
    tex = read_fasta(f"masked_genome/{te_name}_genome.bl")
    tex = tex[tex.qseqid == te_name]
    tex["start"] = tex['sstart'].where(tex['sstart'] <= tex["send"], other=tex['send'])
    tex["stop"] = tex['sstart'].where(tex['sstart'] > tex["send"], other=tex['send'])
    # todo
    save_fasta(tex[["sseqid", "start", "stop"]], f"masked_genome/to_mask_genome_{te_name}.txt")
    plus_tex = tex[tex.sstart < tex.send]
    minus_tex = tex[tex.sstart > tex.send]

    merge_headers = ["sseqid", "sstart", "send"]
    save_fasta(plus_tex[merge_headers].sort_values(by=merge_headers), f"masked_genome/plus_to_mask_genome_{te_name}.bed")
    merge_headers = ["sseqid", "send", "sstart"]
    save_fasta(minus_tex[merge_headers].sort_values(by=merge_headers), f"masked_genome/minus_to_mask_genome_{te_name}.bed")

    os.system(f"bedtools merge -i masked_genome/plus_to_mask_genome_{te_name}.bed -d 200 > masked_genome/plus_to_mask_genome_{te_name}.bed.merged")
    os.system(f"bedtools merge -i masked_genome/minus_to_mask_genome_{te_name}.bed -d 200 > masked_genome/minus_to_mask_genome_{te_name}.bed.merged")
    minus_merged =  pd.read_csv(f"masked_genome/minus_to_mask_genome_{te_name}.bed.merged", sep="\t", header=None)
    plus_merged =  pd.read_csv(f"masked_genome/plus_to_mask_genome_{te_name}.bed.merged", sep="\t", header=None)
    merged_genome = pd.concat(
        [
            minus_merged[
                (minus_merged[2] - minus_merged[1] > from_te)
                & (minus_merged[2] - minus_merged[1] < to_te)
            ],
            plus_merged[
                (plus_merged[2] - plus_merged[1] > from_te)
                & (plus_merged[2] - plus_merged[1] < to_te)
            ],
        ]
    )
    merged_genome["length"] = merged_genome[2] - merged_genome[1]
    save_fasta(merged_genome, f"masked_genome/to_mask_genome_{te_name}.bed.merged")
    os.system(f"bedtools maskfasta -fi {genome_path} -bed masked_genome/to_mask_genome_{te_name}.bed.merged -fo genome/{te_name}_masked_genome")
    return merged_genome
doxi = mask_genome(te_name, te_path, genome_path, True)



Building a new DB, current time: 04/09/2024 06:31:52
New DB name:   /home/kiki/data/genome/genome
New DB title:  genome/DCARv3.4.fa
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data/genome/genome
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 9 sequences in 4.54304 seconds.




In [7]:
for file in ont:
    os.system(f"seqtk seq -a ont/{file}.fastq.gz > ont/{file}.fasta")
#!seqtk seq -a ont/K10B.fastq.gz > ont/K10B.fasta
for file in ont:
    os.system(f"makeblastdb -in ont/{file}.fasta -dbtype nucl -out ont/{file}")
#!makeblastdb -in ont/K10B.fasta -dbtype nucl -out ont/K10B
!mkdir -p blast
for file in ont:
    os.system(f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db ont/{file} -out blast/TE_{file}.bl -dust no -perc_identity 0.9")
#!blastn -num_threads 20 -outfmt 6 -query te/Alex1_Alex3.fasta -db ont/K10B -out blast/Alex_K10B.bl -dust no -perc_identity 0.9

headers = ['qseqid', 'sseqid',  'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'expect value', 'bitscore']
headers_dict = {i[0]:i[1] for i in enumerate(headers)}
data = read_fasta("blast/TE_K10B.bl")

alex1 = data[data.qseqid.str.contains('Alex1')]
plus_alex1 = alex1[alex1.sstart < alex1.send]
minus_alex1 = alex1[alex1.sstart > alex1.send]

alex3 = data[data.qseqid.str.contains('Alex3')]
plus_alex3 = alex3[alex3.sstart < alex3.send]
minus_alex3 = alex3[alex3.sstart > alex3.send]

p_bed_df = plus_alex1[['sseqid', 'sstart', 'send', 'qseqid']].sort_values(by=['sseqid', 'sstart', 'send'])
m_bed_df = minus_alex1[['sseqid', 'send', 'sstart', 'qseqid']].sort_values(by=['sseqid', 'send', 'sstart'])

save_fasta(m_bed_df, 'm_raw_first_blast.bed')
save_fasta(p_bed_df, 'p_raw_first_blast.bed')

!bedtools merge -i p_raw_first_blast.bed -d 1500 -c 4 -o collapse  > p_first_blast.bed.merged
!bedtools merge -i m_raw_first_blast.bed -d 1500 -c 4 -o collapse  > m_first_blast.bed.merged

p_merged = pd.read_csv(
    "p_first_blast.bed.merged", sep="\t", header=None).rename(
    {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
)
m_merged = pd.read_csv(
    "m_first_blast.bed.merged", sep="\t", header=None).rename(
    {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
)

p_filtered_first_bed = p_merged[(p_merged.send - p_merged.sstart) > from_te]
m_filtered_first_bed = m_merged[(m_merged.send - m_merged.sstart) > from_te]

pd.options.mode.copy_on_write = True
p_filtered_first_bed['end'] = p_filtered_first_bed['send'] + 4000
p_filtered_first_bed['start'] = p_filtered_first_bed['sstart'] - 4000
save_fasta(p_filtered_first_bed[['sseqid', 'sstart', 'send']], 'p_normal.txt')

m_filtered_first_bed['end'] = m_filtered_first_bed['send'] + 4000
m_filtered_first_bed['start'] = m_filtered_first_bed['sstart'] - 4000

save_fasta(m_filtered_first_bed[['sseqid', 'sstart', 'send']], 'm_normal.txt')

save_fasta(p_filtered_first_bed[['sseqid']], 'p_seq_list.txt')
save_fasta(m_filtered_first_bed[['sseqid']], 'm_seq_list.txt')

#long
!seqtk subseq ont/K10B.fasta p_seq_list.txt > p_normal.fasta
!seqtk subseq ont/K10B.fasta m_seq_list.txt > m_normal.fasta

!bedtools maskfasta -fi p_normal.fasta -bed p_normal.txt -fo p_Alex_K10B.masked.fasta
!seqkit fx2tab --length --name  p_Alex_K10B.masked.fasta > p_max_lenght.txt
p_max_len = pd.read_csv(
    'p_max_lenght.txt', sep="\t", header=None).rename(
    {0:'sseqid', 1:'max_len'}, axis=1).sort_values(by=['sseqid'])
!bedtools maskfasta -fi m_normal.fasta -bed m_normal.txt -fo m_Alex_K10B.masked.fasta
!seqkit fx2tab --length --name  m_Alex_K10B.masked.fasta > m_max_lenght.txt
m_max_len = pd.read_csv(
    'm_max_lenght.txt', sep="\t", header=None).rename(
    {0:'sseqid', 1:'max_len'}, axis=1).sort_values(by=['sseqid'])

#sseqid bo query to po prostu te w tym przypadku
save_fasta(p_filtered_first_bed[['sseqid', 'start', 'end']], 'p_cutted.txt')
save_fasta(m_filtered_first_bed[['sseqid', 'start', 'end']], 'm_cutted.txt')

p_mask_min = p_filtered_first_bed.start < 0
p_filtered_first_bed.loc[p_mask_min, 'start'] = 0
p_filtered_first_bed = pd.merge(p_filtered_first_bed, p_max_len, how='left', on="sseqid")
p_mask_max = p_filtered_first_bed.end > p_filtered_first_bed.max_len
p_filtered_first_bed.loc[p_mask_max, 'end'] = p_filtered_first_bed.max_len
save_fasta(p_filtered_first_bed[['sseqid', 'start', 'end']], 'p_cutted2.txt')

m_mask_min = m_filtered_first_bed.start < 0
m_filtered_first_bed.loc[m_mask_min, 'start'] = 0
m_filtered_first_bed = pd.merge(m_filtered_first_bed, m_max_len, how='left', on="sseqid")
m_mask_max = m_filtered_first_bed.end > m_filtered_first_bed.max_len
m_filtered_first_bed.loc[m_mask_max, 'end'] = m_filtered_first_bed.max_len
save_fasta(m_filtered_first_bed[['sseqid', 'start', 'end']], 'm_cutted2.txt')

# bedtools getfasta
!seqtk subseq p_Alex_K10B.masked.fasta p_cutted2.txt > p_cutted_ont.fasta
!seqtk subseq m_Alex_K10B.masked.fasta m_cutted2.txt > m_cutted_ont.fasta


[E::stk_seq] failed to open the input file/stream.




Building a new DB, current time: 04/08/2024 21:22:17
New DB name:   /home/kiki/data/ont/K10B
New DB title:  ont/K10B.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data/ont/K10B
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6939343 sequences in 200.464 seconds.




Building a new DB, current time: 04/08/2024 21:25:39
New DB name:   /home/kiki/data/ont/K10f
New DB title:  ont/K10f.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data/ont/K10f
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 4775182 sequences in 145.384 seconds.




Building a new DB, current time: 04/08/2024 21:28:05
New DB name:   /home/kiki/data/ont/LK_1.3
New DB title:  ont/LK_1.3.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B


BLAST options error: File ont/LK_1.3.fasta is empty
FASTA-Reader: Ignoring invalid residues at position(s): On line 109: 14-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 110: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 111: 1-47
FASTA-Reader: Ignoring invalid residues at position(s): On line 114: 45-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 115: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 116: 1-17
FASTA-Reader: Ignoring invalid residues at position(s): On line 142: 15-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 143: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 144: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 145: 1-42
FASTA-Reader: Ignoring invalid residues at position(s): On line 109: 14-60
FASTA-Reader: Ignoring invalid residues at position(s): On line 110: 1-60
FASTA-Reader: Ignoring invalid residues at position(s): 

In [95]:
doxi['from'] = doxi[1] - 2000
doxi['to'] = doxi[2] + 2000

In [102]:
h = [0, "from", "to"]
save_fasta(doxi[h], 'doxi')
!seqtk subseq genome/masked_DCARv3.4.fa doxi > dixi.fasta

In [103]:
!mkdir okno
os.system("makeblastdb -in ont/K10B.fasta -dbtype nucl -out okno/okno")




Building a new DB, current time: 04/08/2024 05:52:47
New DB name:   /home/kiki/data/okno/okno
New DB title:  ont/K10B.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6939343 sequences in 341.898 seconds.




0

In [104]:
os.system("blastn -num_threads 20 -outfmt 6 -query dixi.fasta -db okno/okno -out wuli -perc_identity 0.9")


0

# Second part of analysis

In [12]:
def second_part(blast_db=False):
    pd.options.mode.copy_on_write = True
    if blast_db:
        os.system("makeblastdb -in genome/masked_DCARv3.4.fa  -dbtype nucl -out genome/masked_DCARv3.4")
    #os.system("blastn -num_threads 20 -outfmt 6 -query p_cutted_ont.fasta -db genome/masked_DCARv3.4 -out p_Alex_K10B_genome.bl -perc_identity 0.8 -max_target_seqs 4")
    os.system("blastn -num_threads 20 -outfmt 6 -query p_cutted_ont.fasta -db genome/masked_DCARv3.4 -out p_Alex_K10B_genome.bl -perc_identity 0.8")
    #os.system("blastn -num_threads 20 -outfmt 6 -query m_cutted_ont.fasta -db genome/masked_DCARv3.4 -out m_Alex_K10B_genome.bl -perc_identity 0.8 -max_target_seqs 4")
    os.system("blastn -num_threads 20 -outfmt 6 -query m_cutted_ont.fasta -db genome/masked_DCARv3.4 -out m_Alex_K10B_genome.bl -perc_identity 0.8")

    plus_blast = pd.read_csv("p_Alex_K10B_genome.bl", sep="\t", header=None).rename(headers_dict, axis=1)
    minus_blast = pd.read_csv("m_Alex_K10B_genome.bl", sep="\t", header=None).rename(headers_dict, axis=1)

    p_plus_blast = plus_blast[plus_blast.sstart <= plus_blast.send]
    m_plus_blast = plus_blast[plus_blast.sstart > plus_blast.send]
    
    p_minus_blast = minus_blast[minus_blast.sstart <= minus_blast.send]
    m_minus_blast = minus_blast[minus_blast.sstart > minus_blast.send]

    p_plus = create_report(p_plus_blast)
    m_plus = create_report(m_plus_blast)
    p_minus = create_report(p_minus_blast)
    m_minus = create_report(m_minus_blast)
    
    p_plus.to_csv("p_plus_report", sep='\t', index=False, encoding='utf-8', header=True) # i add header
    m_plus.to_csv("m_plus_report", sep='\t', index=False, encoding='utf-8', header=True) # i add header
    p_minus.to_csv("p_minus_report", sep='\t', index=False, encoding='utf-8', header=True) # i add header
    m_minus.to_csv("m_minus_report", sep='\t', index=False, encoding='utf-8', header=True) # i add header

    p_plus_final = filter_finish(p_plus)
    m_plus_final = filter_finish(m_plus)
    p_minus_final = filter_finish(p_minus)
    m_minus_final = filter_finish(m_minus)
    
    save_fasta(p_plus_final, 'p_plus_final')
    save_fasta(m_plus_final, 'm_plus_final')
    save_fasta(p_minus_final, 'p_minus_final')
    save_fasta(m_minus_final, 'm_minus_final')
    
    final_concat =  pd.concat(
        [
            p_plus_final, 
            m_plus_final, 
            p_minus_final, 
            m_minus_final, 
        ]
    ).sort_values(by=['sseqid', 'start_genome_te'])

    #fin[final_.end_genome_te - ni.start_genome_te > 20]
    return final_concat

oxi = second_part()

In [107]:
oxi.head(50)

,qseqid,sseqid,start_te,end_te,start_genome_te,end_genome_te,qstart1,qend1,sstart1,send1,qstart2,qend2,sstart2,send2,length,length_genome
0,9676ad26-b838-4bf4-b6f3-803f3cb06ff0:8506-20811,DCARv3_Chr1,4000,8307,2884076,2884063,8307,9048,2884063,2883315,1,4000,2888128,2884076,4307,-13
0,2c40afb0-7c90-4ddf-847c-a587e291d4a9:1-9021,DCARv3_Chr1,612,5042,2884076,2884045,5042,9021,2884045,2879983,31,612,2884668,2884076,4430,-31
0,9ebca745-06aa-4187-9303-3fc73853bf74:1-10652,DCARv3_Chr1,2054,6653,32973557,32973561,6653,10652,32973561,32969507,27,2054,32975637,32973557,4599,4
1,98c0c670-f831-449a-a2d7-6b50aad7d341:308-12808,DCARv3_Chr1,4000,8546,45818829,45818832,8546,10780,45818832,45816544,815,4000,45822054,45818829,4546,3
0,aa99dd0d-ba11-4f47-a79e-5dfedc9da5e2:1-10346,DCARv3_Chr2,1669,6354,40807541,40807546,6354,10346,40807546,40811549,28,1669,40805891,40807541,4685,5
3,f4299119-f959-4ec2-b397-a45bdecddf0a:15450-28070,DCARv3_Chr2,3996,8622,49471982,49471985,8622,10010,49471985,49470560,1905,3996,49474048,49471982,4626,3
0,d02bc011-35ab-415e-a9f6-b219b7497dd8:2959-15527,DCARv3_Chr2,4000,8570,52699837,52699836,8570,10315,52699836,52698076,1,4000,52703860,52699837,4570,-1
0,1fe502eb-b72d-4720-8b6b-8905986876f3:1-4932,DCARv3_Chr3,108,4796,55479673,55484403,4796,4922,55484403,55484530,50,108,55479612,55479673,4688,4730
0,328ef23f-1ebb-43b9-a01d-329e58159e42:1655-13665,DCARv3_Chr3,4000,8098,55479676,55483962,8098,12011,55483962,55487910,3,4000,55475646,55479676,4098,4286
2,d6937040-67bd-436b-b6c0-c9b6715e0a4c:1-10613,DCARv3_Chr3,2505,7067,62752814,62752854,7067,8041,62752854,62753828,1425,2505,62751700,62752814,4562,40
